In [1]:
from nba_api.stats.endpoints import PlayerGameLog, TeamGameLog, LeagueGameLog
from nba_api.stats.static import players
from nba_api.stats.static import teams
import numpy
import pandas as pd

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 60)

In [2]:
player_name = 'James Harden'
player = players.find_players_by_full_name(player_name)[0]
player_id = player['id']

In [3]:
game_log = PlayerGameLog(player_id=player_id, season="2023-24").get_data_frames()[0]
league_log = LeagueGameLog(season='2023-24').get_data_frames()[0]
game_log

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22023,201935,0022301184,"APR 12, 2024",LAC vs. UTA,L,10,1,4,0.250,0,2,0.000,2,3,0.667,0,0,0,5,0,0,1,0,4,8,1
1,22023,201935,0022301132,"APR 07, 2024",LAC vs. CLE,W,26,6,11,0.545,4,8,0.500,6,6,1.000,0,5,5,5,0,0,2,3,22,-20,1
2,22023,201935,0022301126,"APR 05, 2024",LAC vs. UTA,W,33,4,10,0.400,2,7,0.286,3,3,1.000,0,10,10,15,0,0,5,1,13,19,1
3,22023,201935,0022301114,"APR 04, 2024",LAC vs. DEN,W,38,6,23,0.261,2,13,0.154,6,7,0.857,1,5,6,8,1,0,1,0,20,4,1
4,22023,201935,0022301101,"APR 02, 2024",LAC @ SAC,L,31,1,7,0.143,1,5,0.200,3,4,0.750,1,4,5,8,1,0,5,0,6,-13,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,22023,201935,0022300024,"NOV 14, 2023",LAC @ DEN,L,35,7,15,0.467,3,8,0.375,4,4,1.000,0,2,2,4,0,0,1,3,21,-3,1
68,22023,201935,0022300179,"NOV 12, 2023",LAC vs. MEM,L,29,4,12,0.333,1,7,0.143,2,2,1.000,2,2,4,3,1,1,1,0,11,-28,1
69,22023,201935,0022300014,"NOV 10, 2023",LAC @ DAL,L,29,3,6,0.500,2,3,0.667,6,6,1.000,0,5,5,3,2,0,2,0,14,-6,1
70,22023,201935,0022300160,"NOV 08, 2023",LAC @ BKN,L,36,4,9,0.444,2,5,0.400,2,2,1.000,0,8,8,5,2,0,5,0,12,-15,1


In [4]:
df = pd.DataFrame(game_log[['GAME_DATE', 'WL', 'MIN', 'FGM','FGA','FG_PCT','FG3M','FG3A','FG3_PCT','FTM','FTA','FT_PCT','OREB','DREB','REB','AST','STL','BLK','TOV','PF','PTS','PLUS_MINUS']])
league_df = pd.DataFrame(league_log[['GAME_DATE', 'WL','MIN','FGM','FGA','FG_PCT','FG3M','FG3A','FG3_PCT','FTM','FTA','FT_PCT','OREB','DREB','REB','AST','STL','BLK','TOV','PF','PTS'	,'PLUS_MINUS']])

In [5]:
df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'], format='mixed')
league_df['GAME_DATE'] = pd.to_datetime(league_df['GAME_DATE'], format='mixed')

In [6]:
df['HOME'] = game_log['MATCHUP'].apply(lambda x: True if 'vs. ' in x else False)
df['TEAM'] = game_log['MATCHUP'].apply(lambda x: x.split()[0])
df['OPPONENT'] = game_log['MATCHUP'].apply(lambda x: x.split()[-1])

league_df['HOME'] = league_log['MATCHUP'].apply(lambda x: True if 'vs. ' in x else False)
league_df['TEAM'] = league_log['MATCHUP'].apply(lambda x: x.split()[0])
league_df['OPPONENT'] = league_log['MATCHUP'].apply(lambda x: x.split()[-1])

In [7]:
merged1 = df.merge(league_df, how = 'left', left_on = ['GAME_DATE', 'TEAM', 'OPPONENT'], right_on=['GAME_DATE', 'TEAM', 'OPPONENT'])
final_df = merged1.merge(league_df, how = 'left', left_on = ['GAME_DATE', 'OPPONENT'], right_on=['GAME_DATE', 'TEAM'])

In [8]:
final_df['date_shifted'] = final_df['GAME_DATE'].shift(1)
final_df['days_rest'] = final_df.date_shifted - final_df.GAME_DATE 
final_df['days_rest'] = final_df['days_rest'].dt.days

In [9]:
final_df['rolling_avg'] = final_df['PTS_x'].rolling(window=5, min_periods=1).mean().values
final_df['rolling_3pa_avg'] = final_df['FG3A_x'].rolling(window=5, min_periods=1).mean().values
final_df['min_rolling'] = final_df['MIN_x'].rolling(window=5, min_periods=1).mean().values
final_df['rolling_team_fta'] = final_df['FTA_y'].rolling(window=5, min_periods=1).mean().values
final_df['rolling_opp_pts'] = final_df['PTS'].rolling(window=5, min_periods=1).mean().values
final_df['rolling_opp_fga'] = final_df['FGA'].rolling(window=5, min_periods=1).mean().values
final_df['rolling_team_fga'] = final_df['FGA_y'].rolling(window=5, min_periods=1).mean().values
final_df['rolling_fga'] = final_df['FGA_x'].rolling(window=5, min_periods=1).mean().values
final_df['rolling_fta'] = final_df['FTA_x'].rolling(window=5, min_periods=1).mean().values
final_df['rolling_team_3pa'] = final_df['FG3A_y'].rolling(window=5, min_periods=1).mean().values
final_df['rolling_team_pts'] = final_df['PTS_y'].rolling(window=5, min_periods=1).mean().values
final_df['rolling_team_fg_pct'] = final_df['FG_PCT_y'].rolling(window=5, min_periods=1).mean().values
final_df['rolling_stls'] = final_df['STL'].rolling(window=5, min_periods=1).mean().values




final_df.drop(columns=['TEAM_y', 'OPPONENT_y', 'HOME_y', 'HOME'])

,GAME_DATE,WL_x,MIN_x,FGM_x,FGA_x,FG_PCT_x,FG3M_x,FG3A_x,FG3_PCT_x,FTM_x,FTA_x,FT_PCT_x,OREB_x,DREB_x,REB_x,AST_x,STL_x,BLK_x,TOV_x,PF_x,PTS_x,PLUS_MINUS_x,HOME_x,TEAM_x,OPPONENT_x,WL_y,MIN_y,FGM_y,FGA_y,FG_PCT_y,FG3M_y,FG3A_y,FG3_PCT_y,FTM_y,FTA_y,FT_PCT_y,OREB_y,DREB_y,REB_y,AST_y,STL_y,BLK_y,TOV_y,PF_y,PTS_y,PLUS_MINUS_y,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,date_shifted,days_rest,rolling_avg,rolling_3pa_avg,min_rolling,rolling_team_fta,rolling_opp_pts,rolling_opp_fga,rolling_team_fga,rolling_fga,rolling_fta,rolling_team_3pa,rolling_team_pts,rolling_team_fg_pct,rolling_stls
0,2024-04-12,L,10,1,4,0.250,0,2,0.000,2,3,0.667,0,0,0,5,0,0,1,0,4,8,True,LAC,UTA,L,240,43,96,0.448,10,39,0.256,13,19,0.684,20,24,44,23,9,7,13,17,109,-1,W,240,41,88,0.466,6,35,0.171,22,25,0.880,14,32,46,26,7,4,18,14,110,1,NaT,NaN,4.00,2.000000,10.00,19.000000,110.0,88.0,96.000000,4.000000,3.00,39.000000,109.0,0.448000,7.000000
1,2024-04-07,W,26,6,11,0.545,4,8,0.500,6,6,1.000,0,5,5,5,0,0,2,3,22,-20,True,LAC,CLE,W,240,41,84,0.488,14,35,0.400,24,24,1.000,11,37,48,26,5,2,16,22,120,2,L,240,42,88,0.477,11,30,0.367,23,27,0.852,7,29,36,27,10,2,11,16,118,-2,2024-04-12,5.0,13.00,5.000000,18.00,21.500000,114.0,88.0,90.000000,7.500000,4.50,37.000000,114.5,0.468000,8.500000
2,2024-04-05,W,33,4,10,0.400,2,7,0.286,3,3,1.000,0,10,10,15,0,0,5,1,13,19,True,LAC,UTA,W,240,51,91,0.560,13,35,0.371,16,18,0.889,8,43,51,32,9,4,15,22,131,29,L,240,35,91,0.385,9,30,0.300,23,29,0.793,14,29,43,20,11,4,15,16,102,-29,2024-04-07,2.0,13.00,5.666667,23.00,20.333333,110.0,89.0,90.333333,8.333333,4.00,36.333333,120.0,0.498667,9.333333
3,2024-04-04,W,38,6,23,0.261,2,13,0.154,6,7,0.857,1,5,6,8,1,0,1,0,20,4,True,LAC,DEN,W,240,34,90,0.378,7,33,0.212,27,32,0.844,7,41,48,22,10,2,7,20,102,2,L,240,36,83,0.434,11,35,0.314,17,25,0.680,6,45,51,27,3,4,14,19,100,-2,2024-04-05,1.0,14.75,7.500000,26.75,23.250000,107.5,87.5,90.250000,12.000000,4.75,35.500000,115.5,0.468500,7.750000
4,2024-04-02,L,31,1,7,0.143,1,5,0.200,3,4,0.750,1,4,5,8,1,0,5,0,6,-13,False,LAC,SAC,L,240,31,76,0.408,11,27,0.407,22,27,0.815,9,29,38,18,6,1,16,21,95,-14,W,240,39,92,0.424,12,42,0.286,19,25,0.760,20,36,56,29,10,6,10,18,109,14,2024-04-04,2.0,13.00,7.000000,27.60,24.000000,107.8,88.4,87.400000,11.000000,4.60,33.800000,111.4,0.456400,8.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2023-11-14,L,35,7,15,0.467,3,8,0.375,4,4,1.000,0,2,2,4,0,0,1,3,21,-3,False,LAC,DEN,L,240,37,86,0.430,13,29,0.448,21,24,0.875,14,28,42,21,10,1,11,24,108,-3,W,240,39,73,0.534,8,23,0.348,25,32,0.781,6,30,36,26,4,2,14,19,111,3,2023-11-17,3.0,16.40,6.000000,34.00,25.000000,105.6,86.0,88.200000,10.600000,6.40,32.800000,110.6,0.450800,5.600000
68,2023-11-12,L,29,4,12,0.333,1,7,0.143,2,2,1.000,2,2,4,3,1,1,1,0,11,-28,True,LAC,MEM,L,240,37,85,0.435,12,34,0.353,15,23,0.652,8,33,41,26,9,3,16,23,101,-4,W,240,37,88,0.420,13,41,0.317,18,24,0.750,11,30,41,25,7,6,16,23,105,4,2023-11-14,2.0,17.00,6.200000,32.40,26.200000,103.4,86.0,86.800000,11.400000,5.60,32.200000,109.6,0.446400,5.800000
69,2023-11-10,L,29,3,6,0.500,2,3,0.667,6,6,1.000,0,5,5,3,2,0,2,0,14,-6,False,LAC,DAL,L,240,43,88,0.489,8,27,0.296,32,36,0.889,19,28,47,21,7,2,14,27,126,-18,W,240,51,98,0.520,19,42,0.452,23,26,0.885,15,26,41,30,7,1,9,23,144,18,2023-11-12,2.0,16.60,6.000000,31.00,27.600000,111.8,87.4,87.400000,11.200000,4.40,32.000000,113.0,0.452400,5.600000
70,2023-11-08,L,36,4,9,0.444,2,5,0.400,2,2,1.000,0,8,8,5,2,0,5,0,12,-15,False,LAC,BKN,L,240,36,91,0.396,8,36,0.222,13,13,1.000,11,40,51,22,10,8,16,15,93,-7,W,240,37,92,0.402,15,41,0.366,11,13,0.846,14,35,49,22,10,10,14,14,100,7,2023-11-10,2.0,16.40,5.400000,32.60,25.200000,112.0,87.8,86.600000,10.60

_x = Player Stats
_y = Team Stats
   = Opp Stats

In [10]:
ohe_df = pd.get_dummies(final_df, columns=['TEAM_x', 'OPPONENT_x', 'WL_x'])
ohe_df

,GAME_DATE,MIN_x,FGM_x,FGA_x,FG_PCT_x,FG3M_x,FG3A_x,FG3_PCT_x,FTM_x,FTA_x,FT_PCT_x,OREB_x,DREB_x,REB_x,AST_x,STL_x,BLK_x,TOV_x,PF_x,PTS_x,PLUS_MINUS_x,HOME_x,WL_y,MIN_y,FGM_y,FGA_y,FG_PCT_y,FG3M_y,FG3A_y,FG3_PCT_y,FTM_y,FTA_y,FT_PCT_y,OREB_y,DREB_y,REB_y,AST_y,STL_y,BLK_y,TOV_y,PF_y,PTS_y,PLUS_MINUS_y,HOME_y,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,HOME,TEAM_y,OPPONENT_y,date_shifted,days_rest,rolling_avg,rolling_3pa_avg,min_rolling,rolling_team_fta,rolling_opp_pts,rolling_opp_fga,rolling_team_fga,rolling_fga,rolling_fta,rolling_team_3pa,rolling_team_pts,rolling_team_fg_pct,rolling_stls,TEAM_x_LAC,OPPONENT_x_ATL,OPPONENT_x_BKN,OPPONENT_x_BOS,OPPONENT_x_CHA,OPPONENT_x_CHI,OPPONENT_x_CLE,OPPONENT_x_DAL,OPPONENT_x_DEN,OPPONENT_x_DET,OPPONENT_x_GSW,OPPONENT_x_HOU,OPPONENT_x_IND,OPPONENT_x_LAL,OPPONENT_x_MEM,OPPONENT_x_MIA,OPPONENT_x_MIL,OPPONENT_x_MIN,OPPONENT_x_NOP,OPPONENT_x_NYK,OPPONENT_x_OKC,OPPONENT_x_ORL,OPPONENT_x_PHI,OPPONENT_x_PHX,OPPONENT_x_POR,OPPONENT_x_SAC,OPPONENT_x_SAS,OPPONENT_x_TOR,OPPONENT_x_UTA,OPPONENT_x_WAS,WL_x_L,WL_x_W
0,2024-04-12,10,1,4,0.250,0,2,0.000,2,3,0.667,0,0,0,5,0,0,1,0,4,8,True,L,240,43,96,0.448,10,39,0.256,13,19,0.684,20,24,44,23,9,7,13,17,109,-1,True,W,240,41,88,0.466,6,35,0.171,22,25,0.880,14,32,46,26,7,4,18,14,110,1,False,UTA,LAC,NaT,NaN,4.00,2.000000,10.00,19.000000,110.0,88.0,96.000000,4.000000,3.00,39.000000,109.0,0.448000,7.000000,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False
1,2024-04-07,26,6,11,0.545,4,8,0.500,6,6,1.000,0,5,5,5,0,0,2,3,22,-20,True,W,240,41,84,0.488,14,35,0.400,24,24,1.000,11,37,48,26,5,2,16,22,120,2,True,L,240,42,88,0.477,11,30,0.367,23,27,0.852,7,29,36,27,10,2,11,16,118,-2,False,CLE,LAC,2024-04-12,5.0,13.00,5.000000,18.00,21.500000,114.0,88.0,90.000000,7.500000,4.50,37.000000,114.5,0.468000,8.500000,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
2,2024-04-05,33,4,10,0.400,2,7,0.286,3,3,1.000,0,10,10,15,0,0,5,1,13,19,True,W,240,51,91,0.560,13,35,0.371,16,18,0.889,8,43,51,32,9,4,15,22,131,29,True,L,240,35,91,0.385,9,30,0.300,23,29,0.793,14,29,43,20,11,4,15,16,102,-29,False,UTA,LAC,2024-04-07,2.0,13.00,5.666667,23.00,20.333333,110.0,89.0,90.333333,8.333333,4.00,36.333333,120.0,0.498667,9.333333,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True
3,2024-04-04,38,6,23,0.261,2,13,0.154,6,7,0.857,1,5,6,8,1,0,1,0,20,4,True,W,240,34,90,0.378,7,33,0.212,27,32,0.844,7,41,48,22,10,2,7,20,102,2,True,L,240,36,83,0.434,11,35,0.314,17,25,0.680,6,45,51,27,3,4,14,19,100,-2,False,DEN,LAC,2024-04-05,1.0,14.75,7.500000,26.75,23.250000,107.5,87.5,90.250000,12.000000,4.75,35.500000,115.5,0.468500,7.750000,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
4,2024-04-02,31,1,7,0.143,1,5,0.200,3,4,0.750,1,4,5,8,1,0,5,0,6,-13,False,L,240,31,76,0.408,11,27,0.407,22,27,0.815,9,29,38,18,6,1,16,21,95,-14,False,W,240,39,92,0.424,12,42,0.286,19,25,0.760,20,36,56,29,10,6,10,18,109,14,True,SAC,LAC,2024-04-04,2.0,13.00,7.000000,27.60,24.000000,107.8,88.4,87.400000,11.000000,4.60,33.800000,111.4,0.456400,8.200000,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [11]:
ohe_df.isna().sum()

GAME_DATE         0
MIN_x             0
FGM_x             0
FGA_x             0
FG_PCT_x          0
                 ..
OPPONENT_x_TOR    0
OPPONENT_x_UTA    0
OPPONENT_x_WAS    0
WL_x_L            0
WL_x_W            0
Length: 115, dtype: int64

In [12]:
last_df = ohe_df.copy()

last_df.drop(columns=['GAME_DATE', 'HOME', 'WL_y', 'HOME_y', 'WL', 'TEAM_y', 'date_shifted', 'OPPONENT_y'], inplace=True)
last_df

,MIN_x,FGM_x,FGA_x,FG_PCT_x,FG3M_x,FG3A_x,FG3_PCT_x,FTM_x,FTA_x,FT_PCT_x,OREB_x,DREB_x,REB_x,AST_x,STL_x,BLK_x,TOV_x,PF_x,PTS_x,PLUS_MINUS_x,HOME_x,MIN_y,FGM_y,FGA_y,FG_PCT_y,FG3M_y,FG3A_y,FG3_PCT_y,FTM_y,FTA_y,FT_PCT_y,OREB_y,DREB_y,REB_y,AST_y,STL_y,BLK_y,TOV_y,PF_y,PTS_y,PLUS_MINUS_y,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,days_rest,rolling_avg,rolling_3pa_avg,min_rolling,rolling_team_fta,rolling_opp_pts,rolling_opp_fga,rolling_team_fga,rolling_fga,rolling_fta,rolling_team_3pa,rolling_team_pts,rolling_team_fg_pct,rolling_stls,TEAM_x_LAC,OPPONENT_x_ATL,OPPONENT_x_BKN,OPPONENT_x_BOS,OPPONENT_x_CHA,OPPONENT_x_CHI,OPPONENT_x_CLE,OPPONENT_x_DAL,OPPONENT_x_DEN,OPPONENT_x_DET,OPPONENT_x_GSW,OPPONENT_x_HOU,OPPONENT_x_IND,OPPONENT_x_LAL,OPPONENT_x_MEM,OPPONENT_x_MIA,OPPONENT_x_MIL,OPPONENT_x_MIN,OPPONENT_x_NOP,OPPONENT_x_NYK,OPPONENT_x_OKC,OPPONENT_x_ORL,OPPONENT_x_PHI,OPPONENT_x_PHX,OPPONENT_x_POR,OPPONENT_x_SAC,OPPONENT_x_SAS,OPPONENT_x_TOR,OPPONENT_x_UTA,OPPONENT_x_WAS,WL_x_L,WL_x_W
0,10,1,4,0.250,0,2,0.000,2,3,0.667,0,0,0,5,0,0,1,0,4,8,True,240,43,96,0.448,10,39,0.256,13,19,0.684,20,24,44,23,9,7,13,17,109,-1,240,41,88,0.466,6,35,0.171,22,25,0.880,14,32,46,26,7,4,18,14,110,1,NaN,4.00,2.000000,10.00,19.000000,110.0,88.0,96.000000,4.000000,3.00,39.000000,109.0,0.448000,7.000000,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False
1,26,6,11,0.545,4,8,0.500,6,6,1.000,0,5,5,5,0,0,2,3,22,-20,True,240,41,84,0.488,14,35,0.400,24,24,1.000,11,37,48,26,5,2,16,22,120,2,240,42,88,0.477,11,30,0.367,23,27,0.852,7,29,36,27,10,2,11,16,118,-2,5.0,13.00,5.000000,18.00,21.500000,114.0,88.0,90.000000,7.500000,4.50,37.000000,114.5,0.468000,8.500000,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
2,33,4,10,0.400,2,7,0.286,3,3,1.000,0,10,10,15,0,0,5,1,13,19,True,240,51,91,0.560,13,35,0.371,16,18,0.889,8,43,51,32,9,4,15,22,131,29,240,35,91,0.385,9,30,0.300,23,29,0.793,14,29,43,20,11,4,15,16,102,-29,2.0,13.00,5.666667,23.00,20.333333,110.0,89.0,90.333333,8.333333,4.00,36.333333,120.0,0.498667,9.333333,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True
3,38,6,23,0.261,2,13,0.154,6,7,0.857,1,5,6,8,1,0,1,0,20,4,True,240,34,90,0.378,7,33,0.212,27,32,0.844,7,41,48,22,10,2,7,20,102,2,240,36,83,0.434,11,35,0.314,17,25,0.680,6,45,51,27,3,4,14,19,100,-2,1.0,14.75,7.500000,26.75,23.250000,107.5,87.5,90.250000,12.000000,4.75,35.500000,115.5,0.468500,7.750000,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
4,31,1,7,0.143,1,5,0.200,3,4,0.750,1,4,5,8,1,0,5,0,6,-13,False,240,31,76,0.408,11,27,0.407,22,27,0.815,9,29,38,18,6,1,16,21,95,-14,240,39,92,0.424,12,42,0.286,19,25,0.760,20,36,56,29,10,6,10,18,109,14,2.0,13.00,7.000000,27.60,24.000000,107.8,88.4,87.400000,11.000000,4.60,33.800000,111.4,0.456400,8.200000,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,35,7,15,0.467,3,8,0.375,4,4,1.000,0,2,2,4,0,0

In [13]:
days_rest_mean = last_df['days_rest'].mean();print(days_rest_mean)
last_df.fillna({'days_rest': days_rest_mean}, inplace=True)


2.2253521126760565


In [14]:
last_df.dropna(inplace=True)

In [15]:
last_df.to_csv('modeling_data.csv')